In [1]:
### Package Setups
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
from tensorflow import keras
import time

from functions_dataCreation import *
from functions_modelArchitectures import *

from tensorflow.keras.callbacks import Callback

import pandas as pd

import wandb
from wandb.keras import WandbCallback

### Data Loading
train = createIODataset(20,'../data/Train')
test = createIODataset(1,'../data/Test')

train = train.repeat(-1)
train = train.shuffle(buffer_size=1024*20,reshuffle_each_iteration=True)
train = train.batch(256,drop_remainder=True)
train = train.prefetch(100)

test = test.repeat(-1)
test = test.shuffle(buffer_size=1024,reshuffle_each_iteration=True)
test = test.batch(1024,drop_remainder=True)
test = test.prefetch(10)

1 Physical GPUs, 1 Logical GPUs


In [2]:
#MobileNetV2 expects pixel vaues in [-1,1], but at this point, the pixel values in your images are in [0-255]. 
 
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


In [3]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = 128
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False,weights='imagenet')

In [4]:
# Freezing (by setting layer.trainable = False) prevents the weights in a given layer from being updated during training.
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
#This feature extractor converts each 128x128x3 image into a 5x5x1280 block of features. Let's see what it does to an example batch of images. (Batch size being 256)

image_batch, label_batch = next(iter(train))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(256, 4, 4, 1280)


In [6]:
# Add a classification head
# To generate predictions from the block of features, average over the spatial 5x5 spatial locations, using a tf.keras.layers.GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image.

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(256, 1280)


In [7]:
# Apply a tf.keras.layers.Dense layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a logit, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

# prediction_layer = tf.keras.layers.Dense(1)
prediction_layer = tf.keras.layers.Dense(1,activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(256, 1)


In [8]:
# Build a model by chaining together the rescaling, base_model and feature extractor layers using the Keras Functional API. As previously mentioned, use training=False as our model contains a BatchNormalization layer.

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [10]:
# Compile the model before training it. Since there are two classes, use a binary cross-entropy loss with from_logits=True since the model provides a linear output.

METRICS = [keras.metrics.Precision(name='precision'),keras.metrics.Recall(name='recall'),keras.metrics.AUC(name='auc'),]

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
            #   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=METRICS)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                

In [11]:
# The 2.5M parameters in MobileNet are frozen, but there are 1.2K trainable parameters in the Dense layer. These are divided between two tf.Variable objects, the weights and biases.

len(model.trainable_variables)


2

In [12]:
run = wandb.init(project="candlestick-CNN", name = 'TLShallow20FilesShuffle20K_AdamLR1e-4' ,reinit= True)

history = model.fit(train
                ,epochs=1000
                ,steps_per_epoch=10240*20/256 #800
                ,verbose=1
                ,validation_data=test                
                ,validation_freq = 10
                ,validation_steps = 100
                ,callbacks=[WandbCallback()]
                )



Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1000
800/800 [==============================] - 63s 79ms/step - loss: 0.5822 - precision: 0.2437 - recall: 0.0473 - auc: 0.5138
Epoch 2/1000
800/800 [==============================] - 64s 79ms/step - loss: 0.5538 - precision: 0.2554 - recall: 0.0169 - auc: 0.5262
Epoch 3/1000
800/800 [==============================] - 63s 79ms/step - loss: 0.5485 - precision: 0.2639 - recall: 0.0065 - auc: 0.5314
Epoch 4/1000
800/800 [==============================] - 63s 79ms/step - loss: 0.5370 - precision: 0.3120 - recall: 0.0025 - auc: 0.5424
Epoch 5/1000
800/800 [==============================] - 64s 79ms/step - loss: 0.5377 - precision: 0.3289 - recall: 0.0011 - auc: 0.5448
Epoch 6/1000
800/800 [==============================] - 63s 79ms/step - loss: 0.5312 - precision: 0.5139 - recall: 8.0458e-04 - auc: 0.5535
Epoch 7/1000
800/800 [==============================] - 63s 79ms/step - loss: 0.5333 - precision: 0.4894 - recall: 4.9494e-04 - auc: 0.5540
Epoch 8/1000
800/800 [==================

KeyboardInterrupt: 